<a href="https://colab.research.google.com/github/Ben93kie/TrackAndTrace/blob/main/DataExploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
import pandas as pd
import os

In [29]:
track_data_path = 'data_sample_AufgDataScience_Case.csv'
event_locations_path = 'event_locations_DataScience_Case.csv'

In [30]:
track_data = pd.read_csv(track_data_path,delimiter=';') 
event_locations = pd.read_csv(event_locations_path,delimiter=';')

In [44]:
track_data.head(10)

,ContainerNumber,ShipmentNumber,PoCreationDate,Material,ShipmentQuantity,Controller,EventName,EventTime,EventMessageTime,EventLocation,VesselName,PortofDischarge,Time2Arrival
0,HLXU8170197,4868820,19.02.18 00:00,A2760101414,72.0,T15,Loading Ship,07.04.18 13:58,08.04.18 03:19,Hafen Rotterdam,NYK ROMULUS,NaN,125.0
1,HLXU8170197,4868820,19.02.18 00:00,A2760101414,72.0,T15,Departure Ship,07.04.18 19:05,09.04.18 16:29,Hafen Rotterdam,NYK ROMULUS,NaN,125.0
2,HLXU8170197,4868820,19.02.18 00:00,A2760101414,72.0,T15,Unloading Ship,21.04.18 06:19,21.04.18 11:59,Hafen Charleston,NYK ROMULUS,NaN,111.0
3,HLXU8170197,4868820,19.02.18 00:00,A2760101414,72.0,T15,Departure Truck,21.04.18 11:42,21.04.18 13:29,Hafen Charleston,NYK ROMULUS,NaN,111.0
4,HLXU8170197,4868820,19.02.18 00:00,A2760101414,72.0,T15,Arrival Truck,08.05.18 18:12,23.05.18 22:04,Hafen Charleston,NYK ROMULUS,NaN,94.0
5,HLXU8170197,4868820,19.02.18 00:00,A2760101414,72.0,T15,Arrival Truck,08.05.18 18:12,23.05.18 17:32,Hafen Charleston,NYK ROMULUS,NaN,94.0
6,HLXU8170197,4868820,19.02.18 00:00,A2760101414,72.0,T15,Loading Truck,16.07.18 14:15,16.07.18 16:47,Consolidation Center Speyer,NYK ROMULUS,NaN,25.0
7,HLXU8170197,4868820,19.02.18 00:00,A2760101414,72.0,T15,Arrival Truck,24.07.18 02:03,24.07.18 05:11,Hafen Antwerpen,NYK ROMULUS,NaN,17.0
8,HLXU8170197,4868820,19.02.18 00:00,A2760101414,72.0,T15,Arrival Ship,02.08.18 21:54,03.08.18 01:03,Bahnterminal Savannah,NYK ROMULUS,NaN,7.0
9,HLXU8170197,4868820,19.02.18 00:00,A2760101414,72.0,T15,Registration Yard,10.08.18 16:18,10.08.18 23:18,Werk Tuscaloosa,NYK ROMULUS,NaN,0.0


In [31]:
event_locations = event_locations.set_index('EventLocation')

In [32]:
event_locations

,Bezeichnung
EventLocation,
urn:jaif:id:loc:26LNLRTM,Hafen Rotterdam
urn:jaif:id:loc:26LUSCHS,Hafen Charleston
urn:jaif:id:loc:26LDESPE,Consolidation Center Speyer
urn:jaif:id:loc:26LBEANR,Hafen Antwerpen
urn:jaif:id:loc:26LUSSAV,Bahnterminal Savannah
urn:jaif:id:loc:25LUN498999044W0138,Werk Tuscaloosa
urn:jaif:id:loc:26LDEBRV,Hafen Bremerhaven
urn:jaif:id:loc:26LDEHAM,Hafen Hamburg
urn:jaif:id:loc:26LMXATM,Hafen Altamira


In [33]:
event_loc_to_name=event_locations.to_dict()

In [72]:
event_loc_to_name = event_loc_to_name['Bezeichnung']


KeyError: ignored

In [73]:
event_loc_to_name[float('NaN')]=float('NaN')

In [74]:
event_loc_to_name

{'urn:jaif:id:loc:26LNLRTM': 'Hafen Rotterdam',
 'urn:jaif:id:loc:26LUSCHS': 'Hafen Charleston',
 'urn:jaif:id:loc:26LDESPE': 'Consolidation Center Speyer',
 'urn:jaif:id:loc:26LBEANR': 'Hafen Antwerpen',
 'urn:jaif:id:loc:26LUSSAV': 'Bahnterminal Savannah',
 'urn:jaif:id:loc:25LUN498999044W0138': 'Werk Tuscaloosa',
 'urn:jaif:id:loc:26LDEBRV': 'Hafen Bremerhaven',
 'urn:jaif:id:loc:26LDEHAM': 'Hafen Hamburg',
 'urn:jaif:id:loc:26LMXATM': 'Hafen Altamira',
 'urn:jaif:id:loc:26LMXAGU': 'Hafen Aguascalientes',
 nan: nan}

In [75]:
track_data.replace({"EventLocation" : event_loc_to_name,"PortofDischarge" : event_loc_to_name},inplace=True)

In [76]:
track_data.head(30)

,ContainerNumber,ShipmentNumber,PoCreationDate,Material,ShipmentQuantity,Controller,EventName,EventTime,EventMessageTime,EventLocation,VesselName,PortofDischarge,Time2Arrival
0,HLXU8170197,4868820,19.02.18 00:00,A2760101414,72.0,T15,Loading Ship,07.04.18 13:58,08.04.18 03:19,Hafen Rotterdam,NYK ROMULUS,NaN,125.0
1,HLXU8170197,4868820,19.02.18 00:00,A2760101414,72.0,T15,Departure Ship,07.04.18 19:05,09.04.18 16:29,Hafen Rotterdam,NYK ROMULUS,NaN,125.0
2,HLXU8170197,4868820,19.02.18 00:00,A2760101414,72.0,T15,Unloading Ship,21.04.18 06:19,21.04.18 11:59,Hafen Charleston,NYK ROMULUS,NaN,111.0
3,HLXU8170197,4868820,19.02.18 00:00,A2760101414,72.0,T15,Departure Truck,21.04.18 11:42,21.04.18 13:29,Hafen Charleston,NYK ROMULUS,NaN,111.0
4,HLXU8170197,4868820,19.02.18 00:00,A2760101414,72.0,T15,Arrival Truck,08.05.18 18:12,23.05.18 22:04,Hafen Charleston,NYK ROMULUS,NaN,94.0
5,HLXU8170197,4868820,19.02.18 00:00,A2760101414,72.0,T15,Loading Truck,16.07.18 14:15,16.07.18 16:47,Consolidation Center Speyer,NYK ROMULUS,NaN,25.0
6,HLXU8170197,4868820,19.02.18 00:00,A2760101414,72.0,T15,Arrival Truck,24.07.18 02:03,24.07.18 05:11,Hafen Antwerpen,NYK ROMULUS,NaN,17.0
7,HLXU8170197,4868820,19.02.18 00:00,A2760101414,72.0,T15,Arrival Ship,02.08.18 21:54,03.08.18 01:03,Bahnterminal Savannah,NYK ROMULUS,NaN,7.0
8,HLXU8170197,4868820,19.02.18 00:00,A2760101414,72.0,T15,Registration Yard,10.08.18 16:18,10.08.18 23:18,Werk Tuscaloosa,NYK ROMULUS,NaN,0.0
9,HLXU8170197,4868820,19.02.18 00:00,A2760101414,72.0,T15,Goods Receipt Dock,10.08.18 20:50,11.08.18 03:50,Werk Tuscaloosa,NYK ROMULUS,NaN,0.0


#remove duplicate row 4 and 5

In [77]:
track_data.drop_duplicates(subset=["ContainerNumber","ShipmentNumber","PoCreationDate","Material","ShipmentQuantity","Controller","EventName","EventTime","EventLocation","VesselName","PortofDischarge","Time2Arrival"],inplace=True,ignore_index=True)

In [78]:
track_data.head(10)

,ContainerNumber,ShipmentNumber,PoCreationDate,Material,ShipmentQuantity,Controller,EventName,EventTime,EventMessageTime,EventLocation,VesselName,PortofDischarge,Time2Arrival
0,HLXU8170197,4868820,19.02.18 00:00,A2760101414,72.0,T15,Loading Ship,07.04.18 13:58,08.04.18 03:19,Hafen Rotterdam,NYK ROMULUS,NaN,125.0
1,HLXU8170197,4868820,19.02.18 00:00,A2760101414,72.0,T15,Departure Ship,07.04.18 19:05,09.04.18 16:29,Hafen Rotterdam,NYK ROMULUS,NaN,125.0
2,HLXU8170197,4868820,19.02.18 00:00,A2760101414,72.0,T15,Unloading Ship,21.04.18 06:19,21.04.18 11:59,Hafen Charleston,NYK ROMULUS,NaN,111.0
3,HLXU8170197,4868820,19.02.18 00:00,A2760101414,72.0,T15,Departure Truck,21.04.18 11:42,21.04.18 13:29,Hafen Charleston,NYK ROMULUS,NaN,111.0
4,HLXU8170197,4868820,19.02.18 00:00,A2760101414,72.0,T15,Arrival Truck,08.05.18 18:12,23.05.18 22:04,Hafen Charleston,NYK ROMULUS,NaN,94.0
5,HLXU8170197,4868820,19.02.18 00:00,A2760101414,72.0,T15,Loading Truck,16.07.18 14:15,16.07.18 16:47,Consolidation Center Speyer,NYK ROMULUS,NaN,25.0
6,HLXU8170197,4868820,19.02.18 00:00,A2760101414,72.0,T15,Arrival Truck,24.07.18 02:03,24.07.18 05:11,Hafen Antwerpen,NYK ROMULUS,NaN,17.0
7,HLXU8170197,4868820,19.02.18 00:00,A2760101414,72.0,T15,Arrival Ship,02.08.18 21:54,03.08.18 01:03,Bahnterminal Savannah,NYK ROMULUS,NaN,7.0
8,HLXU8170197,4868820,19.02.18 00:00,A2760101414,72.0,T15,Registration Yard,10.08.18 16:18,10.08.18 23:18,Werk Tuscaloosa,NYK ROMULUS,NaN,0.0
9,HLXU8170197,4868820,19.02.18 00:00,A2760101414,72.0,T15,Goods Receipt Dock,10.08.18 20:50,11.08.18 03:50,Werk Tuscaloosa,NYK ROMULUS,NaN,0.0


group based on how the transport method is\
welche time zone? -> assume that it is german time only. do not consider winter/summer time\
already sorted?\
portofdischarge nan, when two different ports are reached for same shipmentnumber? -> no, e.g. 5083298 \
probably make prediction from specific point a to specific point b\
timetoarrival sometimes NaN, e.g. 5472546\
new scheduling meaning? e.g 5681685

In [60]:
def get_single_shipment(ShipmentNumber):
  return track_data[track_data['ShipmentNumber']==ShipmentNumber]

In [69]:
def get_random_shipment_number():
  return int(track_data.sample()['ShipmentNumber'])

In [86]:
get_single_shipment(get_random_shipment_number())

,ContainerNumber,ShipmentNumber,PoCreationDate,Material,ShipmentQuantity,Controller,EventName,EventTime,EventMessageTime,EventLocation,VesselName,PortofDischarge,Time2Arrival
32806,HLXU8213051,5681685,19.03.19 00:00,A1662707202,18.0,T15,Arrival Truck,13.04.19 06:07,13.04.19 09:02,Hafen Hamburg,PHILADELPHIA EXPRESS,Hafen Charleston,NaN
32807,HLXU8213051,5681685,13.03.19 00:00,A1662707202,18.0,T15,Arrival Truck,13.04.19 06:07,13.04.19 09:02,Hafen Hamburg,PHILADELPHIA EXPRESS,Hafen Charleston,NaN
32808,HLXU8213051,5681685,19.03.19 00:00,A1662708602,12.0,T15,Arrival Truck,13.04.19 06:07,13.04.19 09:02,Hafen Hamburg,PHILADELPHIA EXPRESS,Hafen Charleston,NaN
32809,HLXU8213051,5681685,19.03.19 00:00,A1662707202,18.0,T15,Loading Ship,18.04.19 01:47,18.04.19 10:35,Hafen Hamburg,PHILADELPHIA EXPRESS,Hafen Charleston,NaN
32810,HLXU8213051,5681685,13.03.19 00:00,A1662707202,18.0,T15,Loading Ship,18.04.19 01:47,18.04.19 10:35,Hafen Hamburg,PHILADELPHIA EXPRESS,Hafen Charleston,NaN
32811,HLXU8213051,5681685,19.03.19 00:00,A1662708602,12.0,T15,Loading Ship,18.04.19 01:47,18.04.19 10:35,Hafen Hamburg,PHILADELPHIA EXPRESS,Hafen Charleston,NaN
32812,HLXU8213051,5681685,19.03.19 00:00,A1662707202,18.0,T15,Departure Ship,18.04.19 04:42,18.04.19 10:37,Hafen Hamburg,PHILADELPHIA EXPRESS,Hafen Charleston,NaN
32813,HLXU8213051,5681685,13.03.19 00:00,A1662707202,18.0,T15,Departure Ship,18.04.19 04:42,18.04.19 10:37,Hafen Hamburg,PHILADELPHIA EXPRESS,Hafen Charleston,NaN
32814,HLXU8213051,5681685,19.03.19 00:00,A1662708602,12.0,T15,Departure Ship,18.04.19 04:42,18.04.19 10:37,Hafen Hamburg,PHILADELPHIA EXPRESS,Hafen Charleston,NaN
32815,HLXU8213051,5681685,19.03.19 00:00,A1662707202,18.0,T15,New Scheduling,27.04.19 06:30,13.04.19 02:39,Hafen Charleston,PHILADELPHIA EXPRESS,Hafen Charleston,NaN


many NaN values

In [85]:
track_data[track_data["Time2Arrival"].isna()]

,ContainerNumber,ShipmentNumber,PoCreationDate,Material,ShipmentQuantity,Controller,EventName,EventTime,EventMessageTime,EventLocation,VesselName,PortofDischarge,Time2Arrival
359,BMOU5574791,4888130,13.03.18 00:00,A1662707202,150.0,T15,Departure Ship,14.04.18 22:30,15.04.18 12:23,Hafen Bremerhaven,NYK DAEDALUS,NaN,NaN
360,BMOU5574791,4888130,13.03.18 00:00,A1662707202,150.0,T15,Unloading Ship,25.04.18 19:50,25.04.18 21:40,Hafen Charleston,NYK DAEDALUS,NaN,NaN
361,BMOU5574791,4888130,13.03.18 00:00,A1662707202,150.0,T15,Departure Truck,25.04.18 20:22,25.04.18 22:30,Hafen Charleston,NYK DAEDALUS,NaN,NaN
362,BMOU5574791,4888130,13.03.18 00:00,A1662707202,150.0,T15,Arrival Truck,10.05.18 17:23,23.05.18 22:04,Hafen Charleston,NYK DAEDALUS,NaN,NaN
363,BMOU5574791,4888130,13.03.18 00:00,A1662707202,150.0,T15,Loading Truck,18.07.18 06:09,18.07.18 08:31,Consolidation Center Speyer,NYK DAEDALUS,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
34268,BMOU4035985,5721724,28.03.19 00:00,A6540107406,18.0,T15,Arrival Ship,19.05.19 04:24,21.05.19 12:57,Hafen Charleston,CHARLESTON EXPRESS,Hafen Charleston,NaN
34269,BMOU4035985,5721724,28.03.19 00:00,A6540107406,18.0,T15,Unloading Ship,19.05.19 14:40,21.05.19 12:58,Hafen Charleston,CHARLESTON EXPRESS,Hafen Charleston,NaN
34270,BMOU4035985,5721724,28.03.19 00:00,A6540107406,18.0,T15,Departure Truck,20.05.19 12:57,21.05.19 13:02,Hafen Charleston,CHARLESTON EXPRESS,Hafen Charleston,NaN
34271,BMOU4035985,5721724,28.03.19 00:00,A6540107406,18.0,T15,Registration Yard,23.05.19 05:10,23.05.19 12:10,Werk Tuscaloosa,CHARLESTON EXPRESS,Hafen Charleston,NaN
